In [1]:
import numpy as np
import pandas as pd
import joblib
import datetime
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from hyperopt import hp, anneal, Trials, STATUS_OK
from hyperopt.fmin import fmin
from functools import partial

In [2]:
# Load the datasets
train_df = pd.read_excel("train_data.xlsx")
test_df = pd.read_excel("test_data.xlsx")

In [3]:
# Define feature matrix and target variable
X_train = train_df.drop(['P'], axis=1)
y_train = train_df['P']
X_test = test_df.drop(['P'], axis=1)
y_test = test_df['P']

In [4]:
def model_metrics(model, X, y):
    """Compute R2 score for model evaluation."""
    y_pred = model.predict(X)
    return r2_score(y, y_pred)

In [5]:

def bayes_fmin(X_train, X_test, y_train, y_test, eval_iters=100):
    """Hyperparameter optimization using Bayesian search."""
    def objective(params):
        model = RandomForestRegressor(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            max_features=int(params['max_features']),
            random_state=42
        )
        model.fit(X_train, y_train)
        loss = -model_metrics(model, X_test, y_test)
        return {"loss": loss, "status": STATUS_OK}

    space = {
        'n_estimators': hp.quniform('n_estimators', 10, 200, 1),
        'max_depth': hp.quniform('max_depth', 1, 50, 1),
        'max_features': hp.quniform('max_features', 1, 3, 1),
    }
    best_params = fmin(objective, space, algo=partial(anneal.suggest), max_evals=eval_iters, trials=Trials())
    best_params = {key: int(value) for key, value in best_params.items()}
    return best_params


In [6]:
# Hyperparameter tuning
best_params = bayes_fmin(X_train, X_test, y_train, y_test, 500)
print("Best Hyperparameters:", best_params)

# Train RandomForestRegressor model
model = RandomForestRegressor(**best_params, random_state=42)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

100%|██████████| 500/500 [33:59<00:00,  4.08s/trial, best loss: -0.9645753040140389]
Best Hyperparameters: {'max_depth': 38, 'max_features': 2, 'n_estimators': 168}


In [8]:
# Model Evaluation
metrics = {
    "RMSE_Train": np.sqrt(mean_squared_error(y_train, y_train_pred)),
    "MAE_Train": mean_absolute_error(y_train, y_train_pred),
    "R2_Train": r2_score(y_train, y_train_pred),
    "RMSE_Test": np.sqrt(mean_squared_error(y_test, y_test_pred)),
    "MAE_Test": mean_absolute_error(y_test, y_test_pred),
    "R2_Test": r2_score(y_test, y_test_pred)
}

for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

RMSE_Train: 12.1317
MAE_Train: 6.3428
R2_Train: 0.9951
RMSE_Test: 30.6692
MAE_Test: 16.3403
R2_Test: 0.9646


In [11]:
def compute_bic(model, X, y):
    """Compute Bayesian Information Criterion (BIC) for a regression model."""
    n = X.shape[0]
    y_pred = model.predict(X)
    rss = np.sum((y - y_pred) ** 2)

    # Approximate number of parameters: n_estimators × max_depth
    # This is a simplification — you can refine it later by counting all nodes
    n_estimators = model.get_params()['n_estimators']
    max_depth = model.get_params()['max_depth']
    k = n_estimators * max_depth

    bic = n * np.log(rss / n) + k * np.log(n)
    return bic


In [12]:
bic_train = compute_bic(model, X_train, y_train)
bic_test = compute_bic(model, X_test, y_test)

print(f"BIC_Train: {bic_train:.2f}")
print(f"BIC_Test: {bic_test:.2f}")


BIC_Train: 66789.00
BIC_Test: 55013.91
